In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
import time
from PID import PID

In [ ]:
# -------------------------
# Parameters
# -------------------------
camera_width = 320
camera_height = 240
forward_speed = 0.1
turn_speed = 0.1
turn_duration = 0.4 

# PID coefficients
steering_pid = PID(Kp=0.0001, Ki=0.0, Kd=0.0005, setpoint=camera_width / 2)

# HSV thresholds
lower_green = np.array([35, 60, 60])
upper_green = np.array([85, 255, 255])
lower_white = np.array([0, 0, 150])
upper_white = np.array([180, 60, 255])

# -------------------------
# Hardware
# -------------------------
camera = Camera.instance(width=camera_width, height=camera_height)
robot = Robot()

# -------------------------
# Widgets
# -------------------------
line_mask_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)

robot_stop = True
stop_button = widgets.Button(description="STOP", button_style='danger', layout=widgets.Layout(width='100px', height='50px'))
start_button = widgets.Button(description="START", button_style='success', layout=widgets.Layout(width='100px', height='50px'))

def stop_robot(b):
    global robot_stop
    robot_stop = True
    robot.stop()

def start_robot(b):
    global robot_stop
    robot_stop = False

stop_button.on_click(stop_robot)
start_button.on_click(start_robot)

display(widgets.VBox([
    line_mask_widget,
    widgets.HBox([start_button, stop_button])
]))

# -------------------------
# State Machine
# -------------------------
state = "FOLLOW_LINE"

def follow_black_line(frame):
    global state, robot_stop

    h, w, _ = frame.shape
    # Focus on bottom-middle third
    x1 = w // 3
    x2 = 2 * w // 3
    y1 = int(h * 0.6)
    roi = frame[y1:h, x1:x2]

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    mask_black = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 21, 7
    )
    kernel = np.ones((5,5), np.uint8)
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, kernel)

    M = cv2.moments(mask_black)
    line_found = M["m00"] > 0
    if line_found:
        cx_line = int(M["m10"] / M["m00"]) + x1
        cv2.circle(frame, (cx_line, h//2), 5, (0,0,255), -1)
        steering_pid.setpoint = w / 2
        steering = steering_pid.update(cx_line)
        left_motor = forward_speed - steering
        right_motor = forward_speed + steering
    else:
        left_motor = right_motor = 0

    if robot_stop:
        left_motor = right_motor = 0

    robot.set_motors(left_motor, right_motor)
    debug_vis = cv2.addWeighted(frame, 0.8, cv2.cvtColor(mask_black, cv2.COLOR_GRAY2BGR), 0.3, 0)
    line_mask_widget.value = bgr8_to_jpeg(debug_vis)

# Attach callback
camera.observe(lambda change: follow_black_line(change['new']), names='value')


In [ ]:
robot.stop()
camera.stop()